In [1]:
import pandas as pd

In [5]:
df1 = pd.read_csv('data/lending_club_before_fe.csv')
df1.head()

C:\Users\aarus\AppData\Local\Temp\ipykernel_30564\1203194901.py:1: DtypeWarning: Columns (32,42,101,112,113,114,117,118,119,122) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('data/lending_club_before_fe.csv')


,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,...,disbursement_method,debt_settlement_flag_date,settlement_date,settlement_percentage,settlement_term,target,month_year,unemployment_rate,inflation_rate,fed_funds_rate
0,3600.0,36 months,13.99,123.03,C,C4,10+ years,MORTGAGE,55000.0,Not Verified,...,Cash,NaN,NaN,NaN,NaN,0,2015-12,5.0,237.761,0.24
1,24700.0,36 months,11.99,820.28,C,C1,10+ years,MORTGAGE,65000.0,Not Verified,...,Cash,NaN,NaN,NaN,NaN,0,2015-12,5.0,237.761,0.24
2,20000.0,60 months,10.78,432.66,B,B4,10+ years,MORTGAGE,63000.0,Not Verified,...,Cash,NaN,NaN,NaN,NaN,0,2015-12,5.0,237.761,0.24
3,10400.0,60 months,22.45,289.91,F,F1,3 years,MORTGAGE,104433.0,Source Verified,...,Cash,NaN,NaN,NaN,NaN,0,2015-12,5.0,237.761,0.24
4,11950.0,36 months,13.44,405.18,C,C3,4 years,RENT,34000.0,Source Verified,...,Cash,NaN,NaN,NaN,NaN,0,2015-12,5.0,237.761,0.24


In [6]:
for col in df1.columns:
    isna = df1[col].isna().sum()
    if isna > 0:
        print(f"{col}: {isna}")

emp_length: 78516
zip_code: 1
dti: 374
inq_last_6mths: 1
mths_since_last_delinq: 678761
mths_since_last_record: 1116786
revol_util: 857
next_pymnt_d: 1345310
last_credit_pull_d: 55
collections_12_mths_ex_med: 56
mths_since_last_major_derog: 991593
annual_inc_joint: 1319544
dti_joint: 1319547
verification_status_joint: 1319749
tot_coll_amt: 67527
tot_cur_bal: 67527
open_acc_6m: 807715
open_act_il: 807714
open_il_12m: 807714
open_il_24m: 807714
mths_since_rcnt_il: 821931
total_bal_il: 807714
il_util: 880303
open_rv_12m: 807714
open_rv_24m: 807714
max_bal_bc: 807714
all_util: 807767
total_rev_hi_lim: 67527
inq_fi: 807714
total_cu_tl: 807715
inq_last_12m: 807715
acc_open_past_24mths: 47281
avg_cur_bal: 67549
bc_open_to_buy: 61145
bc_util: 61914
chargeoff_within_12_mths: 56
mo_sin_old_il_acct: 105576
mo_sin_old_rev_tl_op: 67528
mo_sin_rcnt_rev_tl_op: 67528
mo_sin_rcnt_tl: 67527
mort_acc: 47281
mths_since_recent_bc: 60223
mths_since_recent_bc_dlq: 1026321
mths_since_recent_inq: 174073
mths_s

In [7]:
len(df1)

1345350

In [8]:
# Columns with more than 90% of data empty
na90 = []
for col in df1.columns:
    isna = df1[col].isna().sum()
    if isna/len(df1) > 0.9:
        na90.append(col)
        print(f"{col}: {isna}")

next_pymnt_d: 1345310
annual_inc_joint: 1319544
dti_joint: 1319547
verification_status_joint: 1319749
revol_bal_joint: 1326715
sec_app_fico_range_low: 1326714
sec_app_fico_range_high: 1326714
sec_app_earliest_cr_line: 1326714
sec_app_inq_last_6mths: 1326714
sec_app_mort_acc: 1326714
sec_app_open_acc: 1326714
sec_app_revol_util: 1327042
sec_app_open_act_il: 1326714
sec_app_num_rev_accts: 1326714
sec_app_chargeoff_within_12_mths: 1326714
sec_app_collections_12_mths_ex_med: 1326714
sec_app_mths_since_last_major_derog: 1338701
hardship_type: 1339594
hardship_reason: 1339594
hardship_status: 1339594
deferral_term: 1339594
hardship_amount: 1339594
hardship_start_date: 1339594
hardship_end_date: 1339594
payment_plan_start_date: 1339594
hardship_length: 1339594
hardship_dpd: 1339594
hardship_loan_status: 1339594
orig_projected_additional_accrued_interest: 1341589
hardship_payoff_balance_amount: 1339594
hardship_last_payment_amount: 1339594
debt_settlement_flag_date: 1312068
settlement_date: 13

In [9]:
# Checking if columns starting with the following are present in above list
prefixes = ("hardship_", "settlement_", "sec_app_")
[c for c in df1.columns if c.startswith(prefixes)]
for c in df1.columns:
    if c.startswith(prefixes) and c not in na90:
        print(c)

hardship_flag


Dropping columns next_pymnt_d, hardship_, settlement_, debt_settlement_flag_date, orig_projected_additional_accrued_interest.
Reasons
- next_pymnt_d: Next payment due date
- hardship_: Tracks whether the applicant has entered a formal hardship program. These are entered in the middle of the loan repayemnt.
- settlement_: Discount negotitaions done at the end

In [10]:
leaky_col = {'next_pymnt_d', 'deferral_term', 'payment_plan_start_date', 'orig_projected_additional_accrued_interest', 'debt_settlement_flag_date', 'last_credit_pull_d'}
leaky_prefix = ('hardship_', 'settlement_', 'sec_app_')
leaky_suffix = ('_joint',)

col_drop = [c for c in df1.columns
            if c.startswith(leaky_prefix)
            or c.endswith(leaky_suffix) 
            or c in leaky_col]
df1 = df1.drop(columns=col_drop, errors="ignore")

Creating a clean record for mths_since_last_delinq, mths_since_last_record, mths_since_recent_bc_dlq, mths_since_last_major_derog, mths_since_recent_revol_delinq <br>
All these columns indicate applicant's historical financial state <b4>
1 indicates they have history, 0 means they are clean. <br>
filling NaN values with 999. High value indicates lower risk <br>

In [11]:
derog_cols = [
    'mths_since_last_delinq', 
    'mths_since_last_record', 
    'mths_since_recent_bc_dlq', 
    'mths_since_last_major_derog',
    'mths_since_recent_revol_delinq',
]

for col in derog_cols:
    df1[f'has_{col.split("_")[-1]}'] = df1[col].notna().astype(int)
    df1[col] = df1[col].fillna(999)

mths_since_recent_bc represents months since the most recent Bankcard. <br>
1 if present, 0 if NaN <br>
0 would indicate no bankcard history. <br>
Not converting to 999 because it would not make sense in this case.

In [12]:
recency_cols = ['mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl']
for col in recency_cols:
    df1[col] = df1[col].fillna(999)

df1['has_installment_history'] = df1['mo_sin_old_il_acct'].notna().astype(int)
df1['mo_sin_old_il_acct'] = df1['mo_sin_old_il_acct'].fillna(0)
df1['num_rev_tl_bal_gt_0'] = df1['num_rev_tl_bal_gt_0'].fillna(0)

In [13]:
df1['has_recent_bc'] = df1['mths_since_recent_bc'].notna().astype(int)
df1['mths_since_recent_bc'] = df1['mths_since_recent_bc'].fillna(0)

In [14]:
df1['mths_since_recent_inq'].describe()

count    1.171277e+06
mean     6.713175e+00
std      5.849921e+00
min      0.000000e+00
25%      2.000000e+00
50%      5.000000e+00
75%      1.000000e+01
max      2.500000e+01
Name: mths_since_recent_inq, dtype: float64

mths_since_recent_inq indicate applicant's last credit inquiry. NaN here signify no such inquiry, which is good. Such inquiries stay on credit for 24 months and impact score for 12 months. Replacing NaN with 25. 

In [15]:
df1['has_recent_inq'] = df1['mths_since_recent_inq'].notna().astype(int)
df1['mths_since_recent_inq'] = df1['mths_since_recent_inq'].fillna(25.0)

In [16]:
core_missing_features = ['dti', 'revol_util', 'emp_length', 'tot_coll_amt', 'total_rev_hi_lim']

for col in core_missing_features:
    df1[f'{col}_was_missing'] = df1[col].isna().astype(int)
    
    if col == 'emp_length':
        df1[col] = df1[col].fillna(df1[col].mode()[0])
    else:
        df1[col] = df1[col].fillna(df1[col].median())

tot_coll_amt, tot_cur_bal, total_rev_hi_lim, acc_open_past_24mths, avg_cur_bal, bc_open_to_buy, bc_util, mo_sin_old_rev_tl_op, num_bc_sats, num_tl_op_past_12m <br>
Applicant has a thin file, hence filling with median and creating indicator column for imp ones

In [17]:
strat_indicators = [
    'mo_sin_old_rev_tl_op', 
    'tot_cur_bal', 
    'bc_util', 
    'avg_cur_bal', 
    'bc_open_to_buy',
]

for col in strat_indicators:
    df1[f'{col}_is_missing'] = df1[col].isna().astype(int)
    df1[col] = df1[col].fillna(df1[col].median())

df1['is_thin_file'] = df1['tot_hi_cred_lim'].isna().astype(int)
df1['tot_hi_cred_lim'] = df1['tot_hi_cred_lim'].fillna(df1['tot_hi_cred_lim'].median())

counts_to_fill = ['num_bc_sats', 'num_tl_op_past_12m', 'acc_open_past_24mths', 'total_rev_hi_lim', 'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit']
for col in counts_to_fill:
    df1[col] = df1[col].fillna(0)

collections_12_mths_ex_med, chargeoff_within_12_mths, tax_liens, pub_rec_bankruptcies, inq_last_6mths <br>
Filling with 0, median would be messy for them

In [18]:
count_grp = ['collections_12_mths_ex_med', 'chargeoff_within_12_mths', 'tax_liens', 'pub_rec_bankruptcies', 'inq_last_6mths']
neg_history = ['num_accts_ever_120_pd', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'tot_coll_amt']
acc_depth = ['num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_sats', 'mort_acc']

for col in count_grp+neg_history+acc_depth:
    df1[col] = df1[col].fillna(0)

In [19]:
# dropping zip to avoid bias based on economic status
df1.drop(columns = ['zip_code'], inplace = True)

Behavioral Ratios: pct_tl_nvr_dlq (Percent never delinquent), percent_bc_gt_75 (Bankcard accounts over 75% limit) <br>
pct_tl_nvr_dlq with 100, indicating no history of delinquency and percent_bc_gt_75 with 0, indicates no maxed out cards

In [20]:
df1['pct_tl_nvr_dlq'] = df1['pct_tl_nvr_dlq'].fillna(100)
df1['percent_bc_gt_75'] = df1['percent_bc_gt_75'].fillna(0)

open_acc_6m, open_act_il, open_il_12m, open_il_24m, mths_since_rcnt_il, total_bal_il, il_util, open_rv_12m, open_rv_24m, max_bal_bc, all_util, inq_fi, total_cu_tl, inq_last_12m <br>
They were introduced after 2015. Hence data collected before that have NaN values for these columns.<br>
Dropping them will result in loosing lot of data and keep as it is will make things difficult while model building and SHAP analysis.<br>

Creating has_modern_metrics col:
- 1 = Loan is from 2015–2018 (Modern data exists).
- 0 = Loan is pre-2015 (Modern data is missing).

Filling NaN with median and not 0 to avoid teaching model that old is good. Median values will signal no info.

In [21]:
modern_metrics = ['open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m'] 

df1['has_modern_metrics'] = df1['all_util'].notna().astype(int)

for col in modern_metrics:
    median_val = df1[df1['has_modern_metrics'] == 1][col].median()
    df1[col] = df1[col].fillna(median_val)